<a href="https://colab.research.google.com/github/thanhvietnguyen/Titanic-ML-project/blob/master/app/api/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
# Using the BytesIO you create a file object out of the response that you get from GitHub
from io import BytesIO
import requests

app = FastAPI()

# Load mô hình
mLink = 'https://github.com/thanhvietnguyen/Titanic-ML-project/blob/master/models/best_model.pkl?raw=true'
mfile = BytesIO(requests.get(mLink).content)
model = joblib.load(mfile)

class Passenger(BaseModel):
    Pclass: int
    Sex: str
    Age: float
    SibSp: int
    Parch: int
    Fare: float
    Embarked: str

@app.get("/")
def read_root():
    return {"message": "Titanic Prediction API is online!"}

@app.post("/predict")
def predict(passenger: Passenger):
    family_size = passenger.SibSp + passenger.Parch + 1
    input_data = [[
        passenger.Age,
        passenger.Fare,
        family_size,
        passenger.Sex,
        passenger.Embarked,
        passenger.Pclass
    ]]

    pred = model.predict(input_data)[0]
    prob = model.predict_proba(input_data)[0][1]

    return {
        "prediction": int(pred),
        "probability_survival": round(prob, 4)
    }